In [67]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
from sklearn.model_selection import cross_val_score

In [69]:
df1=pd.read_csv('bangalore_house_data_cleaned.csv')
df1.head()

,area_type,location,size,total_sqft,bath,balcony,price,no_of_bedrooms,price_per_sqft
0,Super built-up Area,1st Block Jayanagar,4 BHK,2850.0,4.0,1.0,428.0,4,15017.543860
1,Super built-up Area,1st Block Jayanagar,3 BHK,1630.0,3.0,2.0,194.0,3,11901.840491
2,Super built-up Area,1st Block Jayanagar,3 BHK,1875.0,2.0,3.0,235.0,3,12533.333333
3,Built-up Area,1st Block Jayanagar,3 BHK,1200.0,2.0,0.0,130.0,3,10833.333333
4,Super built-up Area,1st Block Jayanagar,2 BHK,1235.0,2.0,2.0,148.0,2,11983.805668


In [70]:
df2=df1.drop(['size','price_per_sqft'],axis=1)
df2.head()

,area_type,location,total_sqft,bath,balcony,price,no_of_bedrooms
0,Super built-up Area,1st Block Jayanagar,2850.0,4.0,1.0,428.0,4
1,Super built-up Area,1st Block Jayanagar,1630.0,3.0,2.0,194.0,3
2,Super built-up Area,1st Block Jayanagar,1875.0,2.0,3.0,235.0,3
3,Built-up Area,1st Block Jayanagar,1200.0,2.0,0.0,130.0,3
4,Super built-up Area,1st Block Jayanagar,1235.0,2.0,2.0,148.0,2


In [71]:
df2.corr()

,total_sqft,bath,balcony,price,no_of_bedrooms
total_sqft,1.000000,0.751367,0.269483,0.829071,0.717182
bath,0.751367,1.000000,0.276104,0.616951,0.874226
balcony,0.269483,0.276104,1.000000,0.206942,0.269135
price,0.829071,0.616951,0.206942,1.000000,0.574012
no_of_bedrooms,0.717182,0.874226,0.269135,0.574012,1.000000


In [72]:
df3=pd.get_dummies(df2)
df3

,total_sqft,bath,balcony,price,no_of_bedrooms,area_type_Built-up Area,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,location_1st Block Jayanagar,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
0,2850.0,4.0,1.0,428.0,4,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,2.0,194.0,3,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,3.0,235.0,3,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1200.0,2.0,0.0,130.0,3,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1235.0,2.0,2.0,148.0,2,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006,1155.0,2.0,1.0,64.0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7007,1200.0,2.0,3.0,70.0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7008,1800.0,1.0,1.0,200.0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7009,1353.0,2.0,2.0,110.0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [73]:
df3.corr()

,total_sqft,bath,balcony,price,no_of_bedrooms,area_type_Built-up Area,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,location_1st Block Jayanagar,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
total_sqft,1.000000,0.751367,0.269483,0.829071,0.717182,0.030767,-0.022863,0.156516,-0.119200,0.023811,...,0.004703,-0.019873,-0.015116,0.109869,-0.006080,0.002945,-0.038822,-0.007935,-0.016820,0.103969
bath,0.751367,1.000000,0.276104,0.616951,0.874226,0.031725,-0.014601,0.168172,-0.128554,0.019962,...,0.020385,-0.010272,-0.028652,0.074496,-0.017043,0.002975,-0.045933,-0.008799,-0.019367,0.076232
balcony,0.269483,0.276104,1.000000,0.206942,0.269135,-0.057072,-0.000542,-0.155031,0.145014,-0.004712,...,-0.031742,0.021259,-0.043334,0.016366,0.022739,0.019677,-0.012706,0.023846,-0.006791,-0.017725
price,0.829071,0.616951,0.206942,1.000000,0.574012,0.005148,-0.013494,0.193073,-0.120920,0.067363,...,-0.005836,-0.017362,-0.021158,0.070010,-0.007705,-0.013015,-0.029473,-0.016742,-0.002822,0.180082
no_of_bedrooms,0.717182,0.874226,0.269135,0.574012,1.000000,0.052918,-0.006808,0.182834,-0.157280,0.030183,...,0.028860,-0.012677,-0.035360,0.053842,-0.004380,-0.003739,-0.048024,-0.011556,-0.027947,0.098401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
location_Yelahanka,0.002945,0.002975,0.019677,-0.013015,-0.003739,0.001921,-0.008117,-0.023274,0.013966,-0.003523,...,-0.002306,-0.002663,-0.007427,-0.020616,-0.004418,1.000000,-0.006531,-0.003523,-0.008227,-0.047814
location_Yelahanka New Town,-0.038822,-0.045933,-0.012706,-0.029473,-0.048024,0.023507,-0.004269,0.000748,-0.020206,-0.001853,...,-0.001213,-0.001400,-0.003906,-0.010842,-0.002323,-0.006531,1.000000,-0.001853,-0.004327,-0.025146
location_Yelenahalli,-0.007935,-0.008799,0.023846,-0.016742,-0.011556,-0.003013,-0.002303,-0.009314,0.008724,-0.000999,...,-0.000654,-0.000755,-0.002107,-0.005848,-0.001253,-0.003523,-0.001853,1.000000,-0.002334,-0.013564
location_Yeshwanthpur,-0.016820,-0.019367,-0.006791,-0.002822,-0.027947,-0.019332,-0.005377,-0.014584,0.026663,-0.002334,...,-0.001527,-0.001764,-0.004920,-0.013657,-0.002926,-0.008227,-0.004327,-0.002334,1.000000,-0.031674


In [106]:
# get the values for axis ----- categories on x and price on y
x=df3.drop(['price'],axis=1)
y=df3.price

In [107]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [108]:
# multiple linear regression
from sklearn import linear_model
ml_regr = linear_model.LinearRegression()
ml_regr.fit(x_train, y_train)


ml_pred = ml_regr.predict(x_test)
print('MAE for ml >',mean_absolute_error(y_test,ml_pred))

MAE for ml > 17.22036840953774


In [83]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score


def find_best_model_using_gridsearchcv(x_train,y_train):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'Random_forest':{
            'model':RandomForestRegressor(),
            'params':{
                'criterion':('mse','mae'),
                'max_features':('auto','sqrt','log2')
            }
        }
    }
    
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(x_train,y_train)
        print('model-', algo_name,'best_score-', gs.best_score_,'best_params-', gs.best_params_)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(x_train,y_train)

model- linear_regression best_score- 0.8350583407067527 best_params- {'normalize': True}
model- lasso best_score- 0.7352354517203177 best_params- {'alpha': 1, 'selection': 'cyclic'}
model- decision_tree best_score- 0.6972512415984171 best_params- {'criterion': 'mse', 'splitter': 'random'}
model- Random_forest best_score- 0.7441822524659552 best_params- {'criterion': 'mse', 'max_features': 'auto'}


,model,best_score,best_params
0,linear_regression,0.835058,{'normalize': True}
1,lasso,0.735235,"{'alpha': 1, 'selection': 'cyclic'}"
2,decision_tree,0.697251,"{'criterion': 'mse', 'splitter': 'random'}"
3,Random_forest,0.744182,"{'criterion': 'mse', 'max_features': 'auto'}"


In [117]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(x.columns==location)[0][0]

    X = np.zeros(len(x.columns))
    X[0] = sqft
    X[1] = bath
    X[2] = bhk
    if loc_index >= 0:
        X[loc_index] = 1
   # print(X)

    return ml_regr.predict([X])[0]

In [118]:
predict_price('location_1st Phase JP Nagar',1000, 2, 2)

98.26453464275774

In [121]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(ml_regr,f)

In [122]:

import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))